In [1]:
import warnings
warnings.filterwarnings('ignore')

import skmob
from skmob.preprocessing import filtering
from skmob.preprocessing import compression
import geopandas as gpd
from skmob.utils import plot
import folium
from folium import plugins
import pandas as pd
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from stats_utils import *
from skmob.utils.plot import *
from skmob.data.load import *

--------

## Filtriamo le geometrie della tratta Firenze-Lastra a Signa all'interno dell'area della FIPILI

In [2]:
fipili_area_gdf = gpd.read_file("internship/ArealeRistretto_fipili/fipili/fipili_.shp")

In [7]:
fipili_area_gdf['tipo_ele'].unique()

array(['di tronco carreggiata', 'raccordo, bretella, svincolo',
       'di rotatoria'], dtype=object)

In [9]:
fipili_carreggiata_gdf=fipili_area_gdf[fipili_area_gdf['tipo_ele']=='di tronco carreggiata']

In [11]:
import folium
latitude = 43.7696
longitude = 11.2558
map_f = folium.Map(location=[latitude, longitude], zoom_start=8, tiles='CartoDB positron')
def style_function(feature):
    return {
        'fillColor': 'black',  
        'color': 'darkred',    
        'weight': 4,         
        'fillOpacity': 0.8,  
    }
folium.GeoJson(
    fipili_carreggiata_gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['pk_uid', 'tipo_ele'],
        aliases=['ID:', 'Type:']
    )
).add_to(map_f)

map_f.save('FIPILI.html')
map_f

In [10]:
fipili_carreggiata_gdf['geometry'] = fipili_carreggiata_gdf['geometry'].buffer(10)

In [11]:
latitude = 43.7696
longitude = 11.2558
map_f = folium.Map(location=[latitude, longitude], zoom_start=8, tiles='CartoDB positron')

folium.GeoJson(
    fipili_carreggiata_gdf,
    tooltip=folium.GeoJsonTooltip(
        fields=['pk_uid', 'tipo_ele'],
        aliases=['ID:', 'Type:']
    )
).add_to(map_f)


map_f

In [12]:
svincoli=[382792,382793,383348,383860,223649,223602]
fipili_carreggiata_gdf=fipili_carreggiata_gdf[~fipili_carreggiata_gdf['pk_uid'].isin(svincoli)]

In [104]:
fipili_carreggiata_gdf.to_file('internship/ArealeRistretto_FIPILI/Buffer10m/FIPILI_Buffer_10m.shp')

In [14]:
fipili_buffer_gdf=gpd.read_file('internship/ArealeRistretto_FIPILI/Buffer10m/FIPILI_Buffer_10m.shp')

In [16]:
from shapely.ops import unary_union
from shapely.geometry import box
selected_geometries = fipili_buffer_gdf[
    fipili_buffer_gdf['pk_uid'].isin([382845, 382844,223583,223636,392711])]['geometry']

combined_geometry = unary_union(selected_geometries) #uniamo tutte le geometrie selezionate sopra in un'unica geometria

bounding_box = combined_geometry.bounds #questo fa i confini del box (rettangolare) delle geometrie selezionate a mano col min e max e unite
bounding_box_geom = box(*bounding_box)  # Converte il box in un poligono di shapely 

In [17]:
fipili_Firenze_Lastra = fipili_buffer_gdf[fipili_buffer_gdf['geometry'].within(bounding_box_geom)]

In [19]:
latitude = 43.7696
longitude = 11.2558
map_f = folium.Map(location=[latitude, longitude], zoom_start=8, tiles='CartoDB positron')

def style_function(feature):
    return {
        'fillColor': 'black',  
        'color': 'darkred',    
        'weight': 4,         
        'fillOpacity': 0.8,  
    }

folium.GeoJson(
    fipili_Firenze_Lastra,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['pk_uid', 'tipo_ele'],
        aliases=['ID:', 'Type:']
    )
).add_to(map_f)
map_f.save('Firenze_Lastra.html')
map_f

In [18]:
fipili_Firenze_Lastra=fipili_Firenze_Lastra.dissolve()

In [19]:
latitude = 43.7696
longitude = 11.2558
map_f = folium.Map(location=[latitude, longitude], zoom_start=8, tiles='CartoDB positron')

def style_function(feature):
    return {
        'fillColor': 'black',  
        'color': 'darkred',    
        'weight': 4,         
        'fillOpacity': 0.8,  
    }
folium.GeoJson(
    fipili_Firenze_Lastra,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['pk_uid', 'tipo_ele'],
        aliases=['ID:', 'Type:']
    )
).add_to(map_f)


map_f

In [112]:
fipili_Firenze_Lastra.to_file("internship/ArealeRistretto_FIPILI/Buffer10m/FIRENZE_LASTRA.shp")

------

## Facciamo lo stesso per Ginestra-Lastra

In [116]:
from shapely.ops import unary_union
from shapely.geometry import box
selected_geometries = fipili_buffer_gdf[
    fipili_buffer_gdf['pk_uid'].isin([381787,383896,382390,381784])]['geometry']

combined_geometry = unary_union(selected_geometries)
bounding_box = combined_geometry.bounds #questo fa i confini del box delle geometrie selezionate col min e max

bounding_box_geom = box(*bounding_box)  # Converte i confini in un poligono

In [117]:
fipili_Lastra_Ginestra = fipili_buffer_gdf[fipili_buffer_gdf['geometry'].within(bounding_box_geom)]

In [121]:
latitude = 43.7696
longitude = 11.2558
map_f = folium.Map(location=[latitude, longitude], zoom_start=8, tiles='CartoDB positron')

folium.GeoJson(
    fipili_Lastra_Ginestra,
    tooltip=folium.GeoJsonTooltip(
        fields=['pk_uid', 'tipo_ele'],
        aliases=['ID:', 'Type:']
    )
).add_to(map_f)


map_f

In [119]:
fipili_Lastra_Ginestra=fipili_Lastra_Ginestra.dissolve()

In [120]:
fipili_Lastra_Ginestra.to_file("internship/ArealeRistretto_FIPILI/Buffer10m/LASTRA_GINESTRA.shp")